In [2]:
import pandas as pd

# Load the dataset
file_path = r'C:\Users\wziller\Milwaukee Tool\_Global AME\MSOE\CS5610\M6\csc5610base\GroupProjectM6\Awards_Batting_HoF.csv'
fe_batting = pd.read_csv(file_path)

# Perform feature engineering

fe_batting['HallOfFame_df_yearid'] = fe_batting['HallOfFame_df_yearid'].fillna(0).astype(int)

# Group by playerID and aggregate relevant statistics
grouped_df = fe_batting.groupby('playerID').agg(
    years_played=('Batting_df_yearID', lambda x: x.nunique()),
    total_games=('Batting_df_G', 'sum'),
    total_at_bats=('Batting_df_AB', 'sum'),
    total_runs=('Batting_df_R', 'sum'),
    total_hits=('Batting_df_H', 'sum'),
    total_walks=('Batting_df_BB', 'sum'),
    total_doubles=('Batting_df_2B', 'sum'),
    total_triples=('Batting_df_3B', 'sum'),
    total_home_runs=('Batting_df_HR', 'sum'),
    total_RBI=('Batting_df_RBI', 'sum'),
    total_SB=('Batting_df_SB', 'sum'),
    total_CS=('Batting_df_CS', 'sum'),
    total_BB=('Batting_df_BB', 'sum'),
    total_SO=('Batting_df_SO', 'sum'),
    total_IBB=('Batting_df_IBB', 'sum'),
    total_HBP=('Batting_df_HBP', 'sum'),
    total_SH=('Batting_df_SH', 'sum'),
    total_SF=('Batting_df_SF', 'sum'),
    total_GIDP=('Batting_df_GIDP', 'sum'),
    HOF_status=('HallOfFame_df_inducted', 'max'),
    HOF_year=('HallOfFame_df_yearid', 'max'),
    BattingPost_G=('BattingPost_df_G', 'sum'),
    BattingPost_AB=('BattingPost_df_AB', 'sum'),
    BattingPost_R=('BattingPost_df_R', 'sum'),
    BattingPost_H=('BattingPost_df_H', 'sum'),
    BattingPost_2B=('BattingPost_df_2B', 'sum'),
    BattingPost_3B=('BattingPost_df_3B', 'sum'),
    BattingPost_HR=('BattingPost_df_HR', 'sum'),
    BattingPost_RBI=('BattingPost_df_RBI', 'sum'),
    BattingPost_SB=('BattingPost_df_SB', 'sum'),
    BattingPost_CS=('BattingPost_df_CS', 'sum'),
    BattingPost_BB=('BattingPost_df_BB', 'sum'),
    BattingPost_SO=('BattingPost_df_SO', 'sum'),
    BattingPost_IBB=('BattingPost_df_IBB', 'sum'),
    BattingPost_HBP=('BattingPost_df_HBP', 'sum'),
    BattingPost_SH=('BattingPost_df_SH', 'sum'),
    BattingPost_SF=('BattingPost_df_SF', 'sum'),
    BattingPost_GIDP=('BattingPost_df_GIDP', 'sum')
).reset_index()

grouped_df['total_singles'] = grouped_df['total_hits'] - (grouped_df['total_doubles'] + grouped_df['total_triples'] + grouped_df['total_home_runs'])

# # Merge back with the original DataFrame to preserve all columns
# merge_df = pd.merge(grouped_df, grouped_df, on='playerID', how='left')

# Calculate new features
grouped_df['batting_avg'] = grouped_df['total_hits'] / grouped_df['total_at_bats']
grouped_df['OBP'] = (grouped_df['total_hits'] + grouped_df['total_walks']) / (grouped_df['total_at_bats'] + grouped_df['total_walks'])
grouped_df['SLG'] = (
    (grouped_df['total_singles'] - grouped_df['total_doubles'] - grouped_df['total_triples'] - grouped_df['total_home_runs']) +
    (2 * grouped_df['total_doubles']) +
    (3 * grouped_df['total_triples']) +
    (4 * grouped_df['total_home_runs'])
) / grouped_df['total_at_bats']
grouped_df['OPS'] = grouped_df['OBP'] + grouped_df['SLG']


grouped_df[['HOF_status']].astype('category')
grouped_df[['playerID']].astype('string')


grouped_df.to_csv('Award_Batting_HOF_FE_csv.csv', index=False)
grouped_df.to_feather('Award_Batting_HOF_FE.csv')


grouped_df.info()

C:\Users\wziller\AppData\Local\Temp\ipykernel_58676\370782501.py:5: DtypeWarning: Columns (25,26,27,46) have mixed types. Specify dtype option on import or set low_memory=False.
  fe_batting = pd.read_csv(file_path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18727 entries, 0 to 18726
Data columns (total 44 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   playerID          18727 non-null  object 
 1   years_played      18727 non-null  int64  
 2   total_games       18727 non-null  int64  
 3   total_at_bats     18727 non-null  int64  
 4   total_runs        18727 non-null  int64  
 5   total_hits        18727 non-null  int64  
 6   total_walks       18727 non-null  int64  
 7   total_doubles     18727 non-null  int64  
 8   total_triples     18727 non-null  int64  
 9   total_home_runs   18727 non-null  int64  
 10  total_RBI         18727 non-null  int64  
 11  total_SB          18727 non-null  int64  
 12  total_CS          18727 non-null  int64  
 13  total_BB          18727 non-null  int64  
 14  total_SO          18727 non-null  int64  
 15  total_IBB         18727 non-null  int64  
 16  total_HBP         18727 non-null  int64 

In [3]:
grouped_df.head()

,playerID,years_played,total_games,total_at_bats,total_runs,total_hits,total_walks,total_doubles,total_triples,total_home_runs,...,BattingPost_IBB,BattingPost_HBP,BattingPost_SH,BattingPost_SF,BattingPost_GIDP,total_singles,batting_avg,OBP,SLG,OPS
0,aardsda01,9,331,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
1,aaronha01,23,-1879200,1320000,2608800,-1004400,1682400,748800,117600,906000,...,0,0,0,0,18400,-2776800,-0.760909,0.225819,0.700909,0.926728
2,aaronto01,7,181,176,102,216,86,42,6,13,...,0,0,0,0,0,155,1.227273,1.152672,1.409091,2.561763
3,aasedo01,13,448,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000
4,abadan01,3,15,21,1,2,4,0,0,0,...,0,0,0,0,0,2,0.095238,0.240000,0.095238,0.335238


In [4]:
grouped_df.describe()

c:\Program Files\Python311\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Program Files\Python311\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Program Files\Python311\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Program Files\Python311\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,years_played,total_games,total_at_bats,total_runs,total_hits,total_walks,total_doubles,total_triples,total_home_runs,total_RBI,...,BattingPost_IBB,BattingPost_HBP,BattingPost_SH,BattingPost_SF,BattingPost_GIDP,total_singles,batting_avg,OBP,SLG,OPS
count,18727.000000,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,1.872700e+04,...,1.872700e+04,18727.000000,18727.000000,18727.000000,18727.000000,1.872700e+04,1.670400e+04,1.674700e+04,1.670400e+04,1.670400e+04
mean,5.014044,-1.487953e+04,1.324396e+04,3.163116e+04,-1.275887e+04,2.445671e+04,1.226642e+04,2.002405e+03,8.942243e+03,3.059032e+04,...,2.109332e+02,56.456880,81.506969,66.698510,203.900732,-3.596994e+04,inf,inf,inf,inf
std,4.617862,4.199124e+05,2.565213e+05,5.257102e+05,3.190789e+05,3.796602e+05,2.135640e+05,3.722253e+04,1.916433e+05,5.433306e+05,...,9.481874e+03,1321.189992,1548.209050,2099.290066,3550.592596,7.181333e+05,NaN,NaN,NaN,NaN
min,0.000000,-3.235781e+07,-6.892160e+06,-1.646400e+06,-2.750177e+07,-3.286800e+05,0.000000e+00,0.000000e+00,0.000000e+00,-7.570500e+05,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-6.161249e+07,-3.075000e+02,-6.910000e+02,-4.215000e+02,-6.952644e+02
25%,1.000000,7.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.134480e-01,0.000000e+00,9.161932e-02
50%,3.000000,4.700000e+01,2.100000e+01,6.000000e+00,7.000000e+00,4.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,4.000000e+00,1.583156e-01,2.380952e-01,1.730769e-01,4.032648e-01
75%,7.000000,2.085000e+02,1.300000e+02,7.900000e+01,7.900000e+01,5.000000e+01,2.500000e+01,5.000000e+00,5.000000e+00,6.600000e+01,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,4.700000e+01,2.692308e-01,5.000000e-01,3.243243e-01,7.467032e-01
max,27.000000,7.320600e+06,1.146600e+07,2.849666e+07,5.632200e+06,1.788998e+07,1.423649e+07,2.452320e+06,1.805026e+07,3.515299e+07,...,1.215984e+06,82026.000000,87360.000000,231616.000000,171000.000000,2.261952e+06,inf,inf,inf,inf
